![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Noelia's ML_OPS Project!  👻👻

###  Notebook: Reco System - Gradio

![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Import zone and utils function.


![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [1]:
import pickle
import os
import pandas as pd
import numpy as np


def load_models(mypath):

    count = 0
    # How many files?
    for path in os.listdir(mypath):
        if os.path.isfile(os.path.join(mypath, path)):
            count += 1

    models = []
    for i in range(0,count):
        with open(mypath+f"m{i}.pkl", "rb") as f:
            models.append(pickle.load(f))
            
    return models, count


def return_movie_Id(df, movie_name):
    mask = df['title'] == movie_name.strip().lower()
    res = df.loc[mask]
    if not res.empty:
        res = res['movieId'].unique().tolist()
    else:
        res = None
    return res
    

def make_ensemble_predictions(models, count, userId, movieId):
    preds = []
    for i in range(count):
        p1 = models[i].predict(userId, movieId).est
        preds.append(p1)
    

    preds = np.array(preds)
    votes = preds>2.5
    nvotes = np.count_nonzero(votes)
    if nvotes>5:
        return "Recomended Movie"
    else:
        return "not recommended Movie"

# userId = 12840, movieId = 'as230'  true_rating = 4.5  --> recommended
# userId = 16285, movieId = as2269 true_rating = 2.5  --> not recommended

In [2]:
df = pd.read_csv('../data/clean/all_ratings_with_title.csv', index_col=0)
df.head()

,movieId,userId,score,title
0,as680,1,1.0,the english civil war
1,ns2186,1,4.5,latte and the magic waterstone
2,hs2381,1,5.0,la diosa coronada
3,ns3663,1,5.0,"frankenstein’s monster’s monster, frankenstein"
4,as9500,1,5.0,kept woman


![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
### Problem: several movies with same name

![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [3]:
mask = df['title']=='national treasure'
df.loc[mask]['movieId'].unique()

array(['ds552', 'ns7565', 'hs2756'], dtype=object)

In [4]:
return_movie_Id(df, 'national treasure')

['ds552', 'ns7565', 'hs2756']

In [5]:
return_movie_Id(df, 'nicole')

['as7565']

![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
### Gradio Init

![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [6]:
import pickle
import numpy as np
import gradio as gr

mypath = '../data/models/ensamble1/'
models, count = load_models(mypath)

mypath = '../data/clean/'



def make_predictions( userId, movieName):
    
    movieId = return_movie_Id(df, movieName)
    print("movieId (inside make predictions)--> ", movieId)
    print("movieName (inside make predictions)--> ", movieName)
    pred = ""
    if movieId!=None:
        if len(movieId)>=1:
            for i in range(len(movieId)):
                pred += "recomendation number "+str(i+1)+" --> "+make_ensemble_predictions(models, count, userId, movieId[i])+"\n"
        
    else:
        pred = "The movie doesnt exist"
    return pred



# userId=12840,score=4.5, movieId=as230

c:\Users\kuens\miniconda3\envs\venvMLOPS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
### Gradio app

![divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [ ]:
#Create the input component for Gradio since we are expecting 4 inputs

userId_input = gr.Number(label = "Enter the userId")
# employment_input = gr.Number(label= "Enter Employement Status {1:For Employed, 2: For Unemployed}")
move_input = gr.Textbox(label = "Enter the movie")
# account_input = gr.Number(label = "Enter your account Balance:")
# We create the output
output = gr.Textbox()


app = gr.Interface(fn = make_predictions, inputs=[userId_input, move_input], outputs=output)
app.launch()

---
### Some batch test

---

In [ ]:
option1 = {
    'userId':66575,
    'title': 'iliza shlesinger: confirmed kills',
    'movieId':'ns5785',
    'score':1.5
}

option2= {
    'userId':258545,
    'title':'spongebob squarepants',
    'movieId':'as922',
    'score':0.5
}

option3= {
    'userId':118361,
    'title':'nasha (kannada)',
    'movieId':'as8025',
    'score':3.5
}

option4= {
    'userId':7793,
    'title':'men in black ii',
    'movieId':'ns7444',
    'score':5.0
}

In [10]:
df.loc[df['movieId']=='ns7444']

,movieId,userId,score,title
22355,ns7444,253,3.0,men in black ii
46925,ns7444,483,1.0,men in black ii
92898,ns7444,930,4.5,men in black ii
93269,ns7444,931,3.0,men in black ii
97194,ns7444,978,2.5,men in black ii
...,...,...,...,...
10978708,ns7444,123925,5.0,men in black ii
10994987,ns7444,124040,2.0,men in black ii
11008005,ns7444,124162,4.5,men in black ii
11015588,ns7444,124261,4.0,men in black ii
